<!-- python flask 练习 -->


In [1]:
#  -*- coding: utf-8 -*-
#  引入flask框架
from flask import Flask, Response, make_response, jsonify
# 引入python mysql
import pymysql
# 引入python的配置读取模块
import configparser
# 引入json模块
import json
# 引入线程模块 模拟nacos心跳监测
from threading import Timer
# 引入flask_restful模块
from flask_restful import Api, Resource

#  引入数据库连接池
from py_trains.py_trains_db import mysql_connect
#  引入日志模块
import logging
# 引入io模块
import io

In [ ]:
SERVER_ADDRESSES = "10.0.59.135:30848"
NAMESPACE = "slide-dev"
client = nacos.NacosClient(SERVER_ADDRESSES, namespace=NAMESPACE)
nacos_conf = client.get_config("icp-cloud-bda-pre-charging-circuitfault-detection-service", "DEFAULT_GROUP", True,
                               True)
conf = configparser.ConfigParser()
conf.readfp(io.StringIO(nacos_conf))

clientInstance = nacos.NacosClient(SERVER_ADDRESSES, namespace=NAMESPACE, username="icpx-dev", password="icpx-dev@icpx2021")

clientInstance.add_naming_instance("icp-cloud-bda-pre-charging-circuitfault-detection-service", conf.get("app", "host"),
                                   conf.get("flask", "port"), "DEFAULT")


def heartbeatCheck():
    try:
        clientInstance.send_heartbeat("icp-cloud-bda-pre-charging-circuitfault-detection-service", conf.get("app", "host"),
                                    conf.get("flask", "port"), "DEFAULT")
    except Exception as e:
        logging.error('Reason:', e)


class RepeatingTimer(Timer):
    def run(self):
        while not self.finished.is_set():
            self.function(*self.args, **self.kwargs)
            self.finished.wait(self.interval)


t = RepeatingTimer(3.0, heartbeatCheck)
t.start()            

In [4]:
#  创建一个学生类 学生姓名 学生班级 学生年龄
class Student(object):
    def __init__(self, name, age, grade):
        self.name = name
        self.age = age
        self.grade = grade


#  创建一个 小明学生 12 3班 20岁
student = Student('小明', 12, 3)

#  打印小明
print(student.name)





小明


In [3]:
#  引入pymysql
import pymysql
#  创建一个test方法
def test():
    #  创建一个链接
    conn = pymysql.connect(host='dev.mysql.db.icpx.hoteamsoft.com', port=13306, user='qmt_u', password='qmt@2021', database='icpx_qmt', charset='utf8')
    #  创建一个游标
    cursor = conn.cursor()
    #  创建一个sql语句
    sql = "select * from usde_negation"
    #  执行sql语句
    cursor.execute(sql)
    #  获取所有的结果
    result = cursor.fetchall()
    #  关闭游标
    cursor.close()
    #  关闭链接
    conn.close()
    #  返回结果
    return result


if __name__ == '__main__':
    #  调用test方法
    result = test()
    #  打印结果
    print(result)

(('没', 1), ('毫不', 2), ('绝不', 3), ('从不', 4), ('缺少', 5), ('从未有过', 6), ('切莫', 7), ('很少', 8), ('无须', 9), ('不要', 10), ('毫无', 11), ('不必', 12), ('难以', 13), ('别', 14), ('非', 15), ('一无', 16), ('停止', 17), ('否', 18), ('未', 19), ('尚无', 20), ('拒绝', 21), ('放下', 22), ('未尝', 23), ('不该', 24), ('扼杀', 25), ('木有', 26), ('莫', 27), ('并未', 28), ('不怎么', 29), ('放弃', 30), ('几乎不', 31), ('不曾', 32), ('极少', 33), ('反对', 34), ('反', 35), ('无', 36), ('不是', 37), ('从来不', 38), ('不用', 39), ('不可以', 40), ('请勿', 41), ('永不', 42), ('从未', 43), ('尚未', 44), ('绝非', 45), ('不能', 46), ('不会', 47), ('从没', 48), ('没有', 49), ('决不', 50), ('禁止', 51), ('缺乏', 52), ('不甚', 53), ('杜绝', 54), ('不好', 55), ('甭', 56), ('休想', 57), ('终止', 58), ('并非', 59), ('远非', 60), ('忌', 61), ('未曾', 62), ('不丁点儿', 63), ('聊', 64), ('怎么不', 65), ('毋', 66), ('弗', 67), ('勿', 68), ('不', 69), ('\ufeff不大', 70), ('没怎么', 71))


In [21]:
#连接influxdb数据库
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

if __name__ == '__main__':
    token = "gGa9bGroE0yvYxutyO-Z_B9GgeUDtw_8KONBHzeHwSnmaH_QMvfhDfPvv-WrSiB-C8ea-ITp3D0X-sxHTWFAAA=="
    org = "hoteamsoft"
    bucket = "hoteamsoft"
    url="http://localhost:8086"
    #  创建一个influxdb客户端
    client = influxdb_client.InfluxDBClient(url=url, token=token, org=org, bucket=bucket)
    write_api = client.write_api(write_options=SYNCHRONOUS)
    #  创建一个点
    point = influxdb_client.Point("hoteamsoft").tag("location", "Prague").field("temperature", 25.3)
    #  写入数据
    write_api.write(bucket, org,point)
    # 实例化查询客户端
    query_api = client.query_api()
    # 创建 Flux 查询
    query = 'from(bucket:"hoteamsoft")\
        |> range(start: -10m)\
        |> filter(fn:(r) => r._measurement == "hoteamsoft")\
        |> filter(fn: (r) => r.location == "Prague")\
        |> filter(fn:(r) => r._field == "temperature" )'
    # 向query()方法传递两个命名参数：org和query
    result = query_api.query(org=org, query=query)
    # 创建结果集
    results = []
    for table in result:
        for record in table.records:
            results.append((record.get_field(), record.get_value()))
    # 打印结果
    print(results)



[('temperature', 25.3), ('temperature', 25.3)]
